In [1]:
# This script extracts the title, link, short description, and determines number of keywords detected in each webpage

#import all libraries
import pandas as pd
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import WebDriverWait, Select
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By 
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup as bs
import time, sys, requests, random

In [2]:
# import and authorize gspread  
import gspread
from oauth2client.service_account import ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
google_key_file = '/Users/zacharywong/Documents/ServiceAccountKey-Secret/pelagic-tracker-338302-eaf0e0e671cb.json'
credentials = ServiceAccountCredentials.from_json_keyfile_name(google_key_file, scope)
gc = gspread.authorize(credentials)
# Global variables
columnName = 'Company Name (Leave Blank if Using LinkedIn Company Mining)'
columnEmployee = 'Number of Employees'
columnLocation = 'Company Location'
columnIndustry = 'Company Industry'
inputIndex = 0;

# paths/baseurls
spreadsheet_id = '1vFXonFCyUlEKa1f0s5tvHCKeTek_sAv7rUPYfYss0Qo'
companyFilePath = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/Misc/Companies/Company Web Scrape Tool - Company Names.csv'
driver_path = '/Users/zacharywong/Documents/Work/Portfolio/DigitalHealthWebscrape/chromedriver'
pathtoFile = '/Users/zacharywong/github/zacharywong2023/DigitalHealthWebscrape/CSV Files/'
googleurl = 'https://www.google.com/'

# floats/ints
adjustDenominator = 2
maxbackOff = 120
maxResult = 1
waitTime = 5
waitRun = 0.3

#input lists
names = []
industries = []
locations = []
employees = []
# bools
useURL = False
useName = False

In [3]:
# exponential wait backoff algorithm 
def wait():
    pass
    global waitRun
    sleepTime = waitRun + random.uniform(0, 1)
    time.sleep(sleepTime)
    waitRun = waitRun*2
    if (waitRun >= maxbackOff):
        sys.exit("error: read from sheets quota exceeded")
        

In [4]:
# Helper function: reads in values from DigitalHealthWebscrape google sheet
# Need spreadsheet ID and the cell address where the value should be read in 

def readinValue(cellLocation, sheetIndex):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(sheetIndex)
    try:
        value = worksheet.acell(cellLocation).value
    except:
        wait()
    return value

In [5]:
# Read Company names from csv file 
def readCompanies():
    df = pd.read_csv(companyFilePath, usecols = range(0, 4))
    global names, industries, locations, employees
    df[columnName]=df[columnName].fillna(' ')
    df[columnIndustry]=df[columnIndustry].fillna(' ')
    df[columnEmployee]=df[columnEmployee].fillna(' ')
    df[columnLocation]=df[columnLocation].fillna(' ')
    names = df[columnName].tolist()
    industries = df[columnIndustry].tolist()
    employees = df[columnEmployee].tolist()
    locations = df[columnLocation].tolist()
    print(names, industries, employees, locations)
    

In [6]:
# Read keywords for Keyword Detection from spreadsheet
def readKeyWords():
    keywords = []
    row = 2;
    sheetIndex = 1
    isDone = False
    cellLocationColumn = 'B'
    while (isDone == False):
        cellLocationKeyWords = cellLocationColumn + str(row)
        try:
            keyword = readinValue(cellLocationKeyWords, sheetIndex)
            keywords.append(keyword)
            if(keyword == None):
                isDone = True
                break
            else:
                row += 1 
        except:
            wait()
    keywords = keywords[0:len(keywords)-1]
    print('Keywords: ', keywords)
    return keywords

In [7]:
# Read whether to activate Keyword Detection from spreadsheet
def readCalculateLiklihood():
    #print(links, names)
    sheetIndex = 1
    cellLocationLiklihood = 'A2'
    calculateLiklihood = False
    try:
        calculateLiklihoodInput = readinValue(cellLocationLiklihood, sheetIndex)
        if (calculateLiklihoodInput == 'Yes'):
            calculateLiklihood = True
        else:
            calculateLiklihood = False
    except:
        wait()
    return calculateLiklihood

In [8]:
# read all user inputs from spreadsheet
def readInput():
    calculateLiklihood = readCalculateLiklihood()
    keywords = readKeyWords()
    return calculateLiklihood, keywords

In [9]:
# Extract links from 1 page of google search results
def extractLinks(soup):
    links = []
    #Tags and classes
    linksTag = 'div'
    linksClass = 'yuRUbf'
    linksAttr = 'href'
    searchLinks = soup.find_all(linksTag, class_ = linksClass)
    for h in searchLinks:
        link = h.a.get(linksAttr)
        links.append(link)
    return links 

In [10]:
# Extract titles from 1 page of google search results
def extractTitles(soup):
    titles = [] 
    
    titlesClass = 'LC20lb MBeuO DKV0Md'
    titlesTag = 'h3'
    searchTitles = soup.find_all(titlesTag, class_= titlesClass)
    for h in searchTitles:
        titles.append(h.text)
    return titles

In [11]:
# Extract texts from 1 page of google search results
def extractTexts(soup):
    texts = []
    textsClass = 'VwiC3b yXK7lf MUxGbd yDYNvb lyLwlc lEBKkf'
    textsTag = 'div'
    searchText = soup.find_all(textsTag, class_= textsClass)
    for h in searchText:
        fullText = h.text
        try:
            splitText = fullText.split('— ', 1)
            text = splitText[1]
            texts.append(text);
        except:
            texts.append(fullText)
    return texts;

In [12]:
# Calculate number of keywords detected in the top result from Google 
def calculateLiklihoods(links, keywords):
    #keywords = ['personalized', 'personalization', 'machine-learning', 'AI', 'Artificial Intelligence', '24/7', 'democratizing']
    liklihoods = []
    detectedWordsAll = []
    ignore = ['[document]', 'a', 'article', 'label', 'script', 'style']
    liklihoodDenom = len(keywords) - adjustDenominator
    
    # for each url, calculate number of words detected
    for url in links: 
        detectedWords = []
        output = ''
        count = 0
        liklihood = 0
       # try: 
        res = requests.get(url, headers={'User-Agent': 'Mozilla/5.0'})
        html_page = res.content
        soup = bs(html_page, 'html.parser')
        text = soup.find_all(text=True)
        for t in text:
            if t.parent.name not in ignore:
                output += '{} '.format(t)
        outputSub = output.split(' ')

        # check if each word matches any in keywords list 
        for word in keywords: 
            if (word in outputSub or word.capitalize() in outputSub or word + '\n' in outputSub):
                detectedWords.append(word)
                count +=1 
        liklihood = round((count / liklihoodDenom), 2)
        #print(liklihood)
        liklihoods.append(liklihood)
        detectedWordsAll.append(detectedWords)
        return liklihoods, detectedWordsAll
        #except: 
         #   print('Skipped: cannot verify SSL')
         #   detectedWordsAll.append('None')
         #   liklihoods.append('None')
         #   return liklihoods, detectedWordsAll
        

In [13]:
# Search each company name on Google 
def searchGoogle(index, googleurl, useURL, waitTime, driver):
    websiteName = names[index]
    driver.get(googleurl)
    searchBar = driver.find_element(By.NAME, 'q')
    useURL = False; 
    useName = True; 
    if (useURL):
        query = "site: " + siteURL
        print('query: ' + query)
        try:
            searchBar.send_keys(query)
            searchBar.send_keys('\n')
        except Exception as e : 
            WebDriverWait(driver, waitTime).until(EC.presence_of_element_located((By.NAME, 'q')))
            searchBar.send_keys(query)
            searchBar.send_keys('\n')
    else:
        query = websiteName
        print('query: ' + query)
        searchBar.send_keys(query)
        searchBar.send_keys('\n')
        

In [14]:
# capture links, header, and text
# pageInfo is a list of dictionaries for each page with keys/value pairs: header, link, text
# extract and load each page of results to pageInfo 
def parseHTML(driver, calculateLiklihood, keywords, maxResult, pageInfo):
    soup = bs(driver.page_source, 'html.parser')
    links = extractLinks(soup);
    texts = extractTexts(soup);
    titles = extractTitles(soup);
    if (calculateLiklihood):
        liklihoods, detectedWordsAll = calculateLiklihoods(links[0:1], keywords)
        pageInfo = addToPageInfo(links, texts, maxResult, pageInfo, liklihoods, detectedWordsAll)
    else:
        pageInfo = addToPageInfo(links, texts, maxResult, pageInfo, liklihoods = None, detectedWordsAll = None)
    return pageInfo

In [15]:
# helper function to add 1 page of results to pageInfo list
def addToPageInfo(links, texts, maxResult, pageInfo, liklihoods, detectedWordsAll):
    index = 0;
    while (index < maxResult):
        #create new dictionary of each search results' percentage of detected words, title, link, text, and detected words
        if (liklihoods == None):
            pageInfo.append({columnIndustry: industries[inputIndex], columnLocation: locations[inputIndex], columnEmployee: employees[inputIndex], "Name": names[inputIndex], "Link": links[index], "About": texts[index]})
        else:
            detectedWords = str(detectedWordsAll[index])
            pageInfo.append({"Name": names[inputIndex], "Percentage of Keywords Detected": liklihoods[index], "Detected KeyWords": detectedWords, columnIndustry: industries[inputIndex], columnLocation: locations[inputIndex], columnEmployee: employees[inputIndex], \
                              "About": texts[index], "Link": links[index]})
        print("Percentage of Keywords Detected: ", liklihoods[index])
        print("Name: " + names[inputIndex] + "Link: " + links[index] + "About: " +  texts[index] )
        index += 1
    return pageInfo

In [16]:
# export final result to CSV 
def exportCSV(df, pathtoFile, fileName):
    # convert pageInfo to pandas dataframe and export as csv 
    df.to_csv(pathtoFile + fileName)
    

In [17]:
# export final result to spreadsheet
def updateSpreadSheet(df, sheetIndex):
    sh = gc.open_by_key(spreadsheet_id)
    worksheet = sh.get_worksheet(6)
    worksheet.clear()
    worksheet.update([df.columns.values.tolist()] + df.values.tolist())
    

In [18]:
# turn dictionary of results ot pandas dataframe to prepare for export to CSV and spreadsheet
def exportDeliverables(pageInfo, pathtoFile, fileName, calculateLiklihood):
    df = pd.DataFrame(pageInfo)
    if (calculateLiklihood):
    #    df = df.sort_values(by = ['Percentage of Keywords Detected'], ascending=False)
        df['Percentage of Keywords Detected'] = df['Percentage of Keywords Detected'] * 100
    sheetIndex = 3
    exportCSV(df, pathtoFile, fileName)
    updateSpreadSheet(df, sheetIndex)
    return df 

In [19]:
# run all helper functions to scrape info from Google 
def runExtraction():
    index = 0
    readCompanies()
    calculateLiklihood, keywords = readInput()
    print('calculateLiklihood: ', calculateLiklihood)
    print("Number of Companies: " + str((len(names))))
    fileName = 'CompanyWebScrape_CSV.csv'
    # Access chromedriver and determine path 
    service = Service(driver_path)
    driver = webdriver.Chrome(service = service)
    
    # list of dictionaries with key/value pairs: title, link, text
    # Contains all information for all search results 
    pageInfo = []
    while (index < (len(names))):
        try: 
            searchGoogle(index, googleurl, useURL, waitTime, driver)
            for page in range(0, 1):
                pageInfo = parseHTML(driver, calculateLiklihood, keywords, maxResult, pageInfo)
                index += 1
            time.sleep(waitRun)
        except: 
            print("error parsing this website")
            index += 1
            print('index: ', index)
        global inputIndex
        inputIndex += 1
    df = exportDeliverables(pageInfo, pathtoFile, fileName, calculateLiklihood)
    return df

In [20]:
# Run program with output on how long program takes 
startTime = time.time()
df = runExtraction()
endTime = time.time()
timeElapsed = endTime - startTime
print('Time Elapsed: ', timeElapsed)
df

['Education at Work', 'PERSONAL FINANCE EDUCATION SERVICES INC', 'MoneyCheck: Personal Finance Education', 'Personal Finance Employee Education Fund ', 'Ramsey Solutions', 'Finance Student Association ', 'Personal Career Guidance & Development', 'Personal Money Store', 'Personal Finance Association (PFA)', 'Personal Finance for Real People', 'Personal Finance Boss', 'Stanford GSB Executive Education', 'Personal Finance Club at OSU', 'Council for Economic Education', 'Center for Global Development', 'Investopedia', 'RSF Social Finance', 'University of Tampa', 'Isenberg School of Management, UMass Amherst', 'Eastern Connecticut State University', 'Post University', 'North Central College', 'Perennial Resources International', 'Rutgers Law School', 'Progrexion', 'SchoolsFirst Federal Credit Union', 'Public Affairs Council', 'Tycoono® ', 'The Pill Club', 'Service Express', 'ConsumerTrack, Inc.', 'WalletHub', 'Freedom Financial Network', 'Salary Finance', 'Empowered Staffing', 'Digit', 'Per

Keywords:  []
calculateLiklihood:  False
Number of Companies: 283
query: Education at Work
error parsing this website
index:  1
query: PERSONAL FINANCE EDUCATION SERVICES INC
error parsing this website
index:  2
query: MoneyCheck: Personal Finance Education
error parsing this website
index:  3
query: Personal Finance Employee Education Fund 
error parsing this website
index:  4
query: Ramsey Solutions
error parsing this website
index:  5
query: Finance Student Association 
error parsing this website
index:  6
query: Personal Career Guidance & Development
error parsing this website
index:  7
query: Personal Money Store
error parsing this website
index:  8
query: Personal Finance Association (PFA)
error parsing this website
index:  9
query: Personal Finance for Real People
error parsing this website
index:  10
query: Personal Finance Boss
error parsing this website
index:  11
query: Stanford GSB Executive Education
error parsing this website
index:  12
query: Personal Finance Club at OSU

error parsing this website
index:  114
query: PAHCOM - Professional Association of Health Care Office Management
error parsing this website
index:  115
query: Workday Learner Community
error parsing this website
index:  116
query: Merit Global Training
error parsing this website
index:  117
query: GAFM Global Academy of Finance & Management ® - ISO 21001 Certified International Board of Standards
error parsing this website
index:  118
query: Technical Safety BC
error parsing this website
index:  119
query: Quake Capital Partners
error parsing this website
index:  120
query: National Institute of Securities Markets
error parsing this website
index:  121
query: 1C Company
error parsing this website
index:  122
query: Octal IT Solution (CMMI Level 3 Appraised)
error parsing this website
index:  123
query: The Futurist Institute
error parsing this website
index:  124
query: Association of Finance & Insurance Professionals
error parsing this website
index:  125
query: Investment Educational

query: National Association of Construction Auditors
error parsing this website
index:  222
query: Pikes Peak Small Business Development Center
error parsing this website
index:  223
query: URBE University
error parsing this website
index:  224
query: University of Michigan-Flint Graduate Programs
error parsing this website
index:  225
query: IMN Solutions
error parsing this website
index:  226
query: Manhattan School of Computer Technology
error parsing this website
index:  227
query: Institute of Fraud and Risk Management
error parsing this website
index:  228
query: International Institute of Certified Forensic Accountants, Inc (USA)
error parsing this website
index:  229
query: Arizona Hotel & Lodging Association
error parsing this website
index:  230
query: Certified Financial Manager: CFM®
error parsing this website
index:  231
query: Association Management Strategies, Inc.
error parsing this website
index:  232
query: Clemson University Master of Professional Accountancy
error p

,Company Industry,Company Location,Number of Employees,Name,Link,About
0,Outsourcing/Offshoring,"Tempe, Arizona",880,Education at Work,https://educationatwork.org/,Education at Work is a U.S.-based provider of ...
1,Individual & Family Services,"Ann Arbor, Michigan",2,PERSONAL FINANCE EDUCATION SERVICES INC,https://www.linkedin.com/company/financialeduc...,"As a Financial Services specialist, I strive t..."
2,Financial Services,"Atlanta, GA",,MoneyCheck: Personal Finance Education,https://www.bbb.org/us/ga/atlanta/profile/busi...,This organization is not BBB accredited. Busin...
3,Financial Services,"Lombard, Illinois",3,Personal Finance Employee Education Fund,https://www.pfeef.org/,Personal Finance Employee Education Fund ... P...
4,Financial Services,"Franklin, TN","1,321",Ramsey Solutions,https://www.ramseysolutions.com/,"Learn to budget, beat debt, save and invest wi..."
...,...,...,...,...,...,...
278,Computer Software,"Alameda, California",1,Stella Sustainability,https://www.stellasustainability.com/,"As your trusted partner, we will help you nego..."
279,Accounting,"Tucson, AZ",1,Central Illinos Chapter of the Institute of Ma...,https://sites.google.com/site/centralillinoisi...,Eric Dubrowski CI Chapter President Central Il...
280,Financial Services,"San Francisco, California",2,"ManageWise, Inc.",https://www.managewise.ca/,"ManageWise is an expert in regional, provincia..."
281,Information Services,"Barrington, NH",,CollegeOne,https://suite.collegeone.net/,CollegeOne Suite. Sign In. Login. Join. Online...
